# Rendering Fpocket results in Nanome

## First we'll analyze results using NGLViewer.

## Then we'll load data into Nanome


In [ ]:
# Set up nanome plugin, which we'll use later
import os
from nanome.beta.redis_interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = os.environ.get("REDIS_HOST")
redis_port = os.environ.get("REDIS_PORT")
redis_password = os.environ.get("REDIS_PASSWORD")
redis_channel = ""  # Use room id code you receive when activating cookbook plugin

plugin = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
plugin.connect()
print('Plugin Connected')

In [ ]:
import torch
import esm

# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0))

# Load the ESMFold model
model = esm.pretrained.esmfold_v1()
print("Loaded the model")

model = model.eval() #.cuda()  # Assuming you have a CUDA-enabled GPU

print("Evaluated the model")
# Example protein sequence
sequence = "MKTVRQERLKSIVRILERSKEP"

# Predict the 3D structure
with torch.no_grad():
    output = model.infer_pdb(sequence)

# Save the predicted structure to a file
with open("result.pdb", "w") as f:
    f.write(output)

print("3D structure prediction saved to result.pdb")


In [ ]:
from nanome.api.structure import Complex
import os
import time
import nglview as nv

# PDB file and fpocket results
pdb_file = "test_data/5ceo.pdb"
fpocket_out = 'test_data/5ceo_fpocket_out'


In [92]:
# Lets view a particular pocket number in NGLView
structure = nv.FileStructure(pdb_file)

representations = [
    {"type": "cartoon", "params": {
        "sele": "protein", "color": "blue"
    }},
    {"type": "ball+stick", "params": {
        "sele": "hetero"
    }},
]
view = nv.NGLWidget(structure, representations=representations)

In [99]:
# Lets view a specific pocket number from the output
import itertools

def view_pocket(view, fpocket_out, pocket_num):
    pocket_file = f'{fpocket_out}/pockets/pocket{pocket_num}_atm.pdb'
    if not os.path.exists(pocket_file):
        print(f"Pocket {pocket_num} not found")
        return False
    pocket_comp = Complex.io.from_pdb(path=pocket_file)
    atom_iter = itertools.chain.from_iterable([res.atoms for res in pocket_comp.residues])
    pocket_atom_nums = sorted([atom.serial for atom in atom_iter])
    view.remove_surface()    
    view.add_surface(selection=pocket_atom_nums, color='red')
    return True

pocket_number = 1
view_pocket(view, fpocket_out, pocket_number)
view

NGLWidget(n_components=1)

In [102]:
def cycle_pocket_surfaces(view, fpocket_out: str):
    # Clear all other surface representations
    i = 1
    while True:
        view.remove_surface()
        pocket_file = f'{fpocket_out}/pockets/pocket{i}_atm.pdb'
        if not os.path.exists(pocket_file):
            break
        success = view_pocket(view, fpocket_out, i)
        if not success:
            break
        # print(f"Pocket {i}")
        time.sleep(1)
        i += 1

cycle_pocket_surfaces(view, fpocket_out)

## Time to load the results into Nanome

In [ ]:
# Create new Complex in your workspace
plugin.send_files_to_load([pdb_file])

In [ ]:
# Get comp with representation data from workspace
comp = plugin.request_workspace().complexes[-1]

In [103]:
from nanome.util import Color
from nanome.api.structure import Complex

def render_pocket_surface(comp, fpocket_out, pocket_num):
    pocket_file = f'{fpocket_out}/pockets/pocket{pocket_num}_atm.pdb'
    if not os.path.exists(pocket_file):
        print(f"Pocket {pocket_num} not found")
        return False
    pocket_comp = Complex.io.from_pdb(path=pocket_file)
    pocket_res_nums = sorted([res.serial for res in pocket_comp.residues])
    for res in comp.residues:
        surface_color = Color.Red() if res.serial in pocket_res_nums else Color.Blue()
        for atom in res.atoms:
            atom.surface_rendering = True
            atom.surface_color = surface_color
    comp._surface_dirty = True

pocket_number = 1
render_pocket_surface(comp, fpocket_out, pocket_number)
plugin.update_structures_deep([comp])


NameError: name 'comp' is not defined